This notebook shows that, when using a clustered splitter, if the clusters explain a part of the variance, using a non-clustered analysis will lead to higher false positive rate than expected.

In particular, we use a clustered splitter and:
* An OLS-clustered robust estimator, we see that it passes the AA test
* A simple t-test, it fails the AA test as it returns a super high false positive rate

In [1]:
from datetime import date

import numpy as np
import pandas as pd
from cluster_experiments import PowerAnalysis, UniformPerturbator, BalancedClusteredSplitter, ExperimentAnalysis
import pandas as pd
from scipy.stats import ttest_ind


# Create fake data
N = 10_000
clusters = [f"Cluster {i}" for i in range(10)]
dates = [f"{date(2022, 1, i):%Y-%m-%d}" for i in range(1, 15)]
df = pd.DataFrame(
    {
        "cluster": np.random.choice(clusters, size=N),
        "date": np.random.choice(dates, size=N),
    }
).assign(
    # Target is a linear combination of cluster and day of week, plus some noise
    cluster_id=lambda df: df["cluster"].astype("category").cat.codes,
    day_of_week=lambda df: pd.to_datetime(df["date"]).dt.dayofweek,
    target=lambda df: df["cluster_id"] + df["day_of_week"] + np.random.normal(size=N),
)


In [2]:
df.head()

,cluster,date,cluster_id,day_of_week,target
0,Cluster 6,2022-01-03,6,0,4.274103
1,Cluster 2,2022-01-07,2,4,4.094490
2,Cluster 1,2022-01-06,1,3,4.898956
3,Cluster 4,2022-01-04,4,1,4.546662
4,Cluster 1,2022-01-01,1,5,6.193957


Some clusters have a higher average outcome than others

In [3]:
df.groupby("cluster").agg({"target": ["mean", "std"]})

target          
                mean       std
cluster                       
Cluster 0   2.893330  2.217032
Cluster 1   3.940817  2.221600
Cluster 2   4.966786  2.235370
Cluster 3   6.012935  2.225931
Cluster 4   7.019475  2.298820
Cluster 5   7.964605  2.254402
Cluster 6   8.871173  2.283362
Cluster 7   9.917630  2.239623
Cluster 8  11.067479  2.235274
Cluster 9  12.053165  2.227960

In [4]:
# Simple t-test to run the analysis
class TTestAnalysis(ExperimentAnalysis):
    def analysis_pvalue(self, df: pd.DataFrame, verbose: bool = True) -> float:
        treatment_data = df.query(f"{self.treatment_col} == 1")[self.target_col]
        control_data = df.query(f"{self.treatment_col} == 0")[self.target_col]
        t_test_results = ttest_ind(treatment_data, control_data, equal_var=False)
        return t_test_results.pvalue

In [5]:
cluster_cols = ["cluster", "date"]

# Right power analysis, we use clustered splitter and ols clustered analysis
pw_right = PowerAnalysis.from_dict(
    {
        "splitter": "clustered_balance",
        "perturbator": "uniform",
        "analysis": "ols_clustered",
        "cluster_cols": cluster_cols,
        "n_simulations": 100,
        "alpha": 0.05,
    },
)

# Wrong power analysis, we use clustered splitter and regular t-test
pw_wrong = PowerAnalysis(
    splitter=BalancedClusteredSplitter(
        cluster_cols=cluster_cols,
    ),
    perturbator=UniformPerturbator(),
    analysis=TTestAnalysis(
        # We pass cluster_cols here, but we don't use them!!!
        cluster_cols=cluster_cols,
    ),
    alpha=0.05,
    n_simulations=100
)



Right way of doing it: in the AA test we get a power similar to the type I error of the test

In [6]:
pw_right.power_analysis(df, average_effect=0.0)

0.04

Wrong way of doing it: the AA test fails, we have too much power

In [7]:
pw_wrong.power_analysis(df, average_effect=0.0)

0.85